In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use the ADK to define an agent and deploy it to Vertex AI Agent Engine
This notebook shows how to use the Agent Framework to define a simple, hello world, agent for a dice rolling game. Then, we deploy the defined agent to the Vertex AI Agent Engine to call and manage the agent in production. In this notebook you will learn how to do the following:

*  Install Agent Framework
*  Define functions as tools
*  Create a "Hello World" agent using the Agent Framework
*  Deploy the agent on Vertex AI Agent Engine
*  Call the Agent Engine to manage and interact with the agent


# Authenticate
You need this to download the SDK from GCS

In [ ]:
from google.colab import auth
auth.authenticate_user()

RELEASE_VERSION='google_adk-0.0.2.dev20250326+nightly740999296'
WHL_FILE = f"{RELEASE_VERSION}-py3-none-any.whl"

# Download and install the ADK and Agent Engine SDK

Download `.whl` file from the shared GCS bucket:

In [ ]:
FOLDER = "gs://agent_framework/history/"

!gsutil cp {FOLDER}{WHL_FILE} .
!pip3 install google_adk-0.0.2.dev*.whl --ignore-requires-python --quiet ## for the google-adk in private preview
!pip3 install "google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226" --force-reinstall --quiet ## for the prebuilt template

Copying gs://agent_framework/history/google_adk-0.0.2.dev20250326+nightly740999296-py3-none-any.whl...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.4/231.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Initialization

In [ ]:
import vertexai
from vertexai import agent_engines

PROJECT_ID = "reasoning-engine-test-1" #@param {type:"string"}
STAGING_BUCKET = "gs://reasoning-engine-test-1-bucket" #@param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location="us-central1",
    staging_bucket=STAGING_BUCKET,
)

# Defining functions
roll_die - rolls dice to create random number

check_prime - checks if a number is prime

In [ ]:
def roll_die(sides: int) -> int:
  """Roll a die and return the rolled result.

  Args:
    sides: The integer number of sides the die has.

  Returns:
    An integer of the result of rolling the die.
  """
  import random

  return random.randint(1, sides)

In [ ]:
def check_prime(nums: list[int]) -> list[str]:
  """Check if a given list of numbers are prime.

  Args:
    nums: The list of numbers to check.

  Returns:
    A str indicating which number is prime.
  """

  primes = set()
  for number in nums:
    number = int(number)
    if number <= 1:
      continue
    is_prime = True
    for i in range(2, int(number**0.5) + 1):
      if number % i == 0:
        is_prime = False
        break
    if is_prime:
      primes.add(number)
  return (
      'No prime numbers found.'
      if not primes
      else f"{', '.join(str(num) for num in primes)} are prime numbers."
  )

# Define an agent in the agent framework

An AI agent reasons, plans, and takes actions. The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Create a simple agent. An agent requires two things: a display name and a set of instructions. The instructions should be detailed, making it clear exactly how the agent should behave.



In [ ]:
from google.adk.agents import Agent

agent = Agent(
    model="gemini-1.5-flash-002",
    name='data_processing_agent',
    instruction="""
      You roll dice and answer questions about the outcome of the dice rolls.
      You can roll dice of different sizes.
      You can use multiple tools in parallel by calling functions in parallel(in one request and in one round).
      The only things you do are roll dice for the user and discuss the outcomes.
      It is ok to discuss previous dice roles, and comment on the dice rolls.
      When you are asked to roll a die, you must call the roll_die tool with the number of sides. Be sure to pass in an integer. Do not pass in a string.
      You should never roll a die on your own.
      When checking prime numbers, call the check_prime tool with a list of integers. Be sure to pass in a list of integers. You should never pass in a string.
      You should not check prime numbers before calling the tool.
      When you are asked to roll a die and check prime numbers, you should always make the following two function calls:
      1. You should first call the roll_die tool to get a roll. Wait for the function response before calling the check_prime tool.
      2. After you get the function response from roll_die tool, you should call the check_prime tool with the roll_die result.
        2.1 If user asks you to check primes based on previous rolls, make sure you include the previous rolls in the list.
      3. When you respond, you must include the roll_die result from step 1.
      You should always perform the previous 3 steps when asking for a roll and checking prime numbers.
      You should not rely on the previous history on prime results.
    """,
    tools=[roll_die, check_prime],
)

app = agent_engines.ADKApp(
    agent=agent,
    enable_tracing=True,
    app_name="test-app", # optional
)

app

In [ ]:
app.register_operations()

{'': ['get_session',
  'list_sessions',
  'create_session',
  'delete_session',
  'list_events'],
 'stream': ['stream_query', 'streaming_agent_run_with_events']}

## Let's try it (locally)

Under the hood, `app: ADKApp` will be using an in-memory session store when running locally.

**Remark**: the operations below (i.e., `.create_session`, `.list_sessions`, `.get_session`, `.stream_query`) are only supported by the `agent_engines.ADKApp` class, not the `google.adk.agents.Agent` class.

In [ ]:
session = app.create_session(user_id="ysian")
session

Session(id='6d94cff6-f9c1-4941-af28-89261ba0f965', app_name='test-app', user_id='ysian', state={}, events=[], last_update_time=1743476013.149413)

In [ ]:
app.list_sessions(user_id="ysian")

ListSessionsResponse(session_ids=['6d94cff6-f9c1-4941-af28-89261ba0f965'])

In [ ]:
session = app.get_session(user_id="ysian", session_id=session.id)
session

Session(id='6d94cff6-f9c1-4941-af28-89261ba0f965', app_name='test-app', user_id='ysian', state={}, events=[], last_update_time=1743476013.149413)

In [ ]:
for event in app.stream_query(
    user_id="ysian",
    session_id=session.id,
    message="roll a 2 sided die",
):
    print(event)

{'parts': [{'function_call': {'id': 'af-54346058-4f09-484c-859f-a7ecf7468acb', 'args': {'sides': 2}, 'name': 'roll_die'}}], 'role': 'model'}
{'parts': [{'function_response': {'id': 'af-54346058-4f09-484c-859f-a7ecf7468acb', 'name': 'roll_die', 'response': {'result': 2}}}], 'role': 'user'}
{'parts': [{'text': 'I rolled a 2.\n'}], 'role': 'model'}


In [ ]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"invocation_id":"14076651604820871801","author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for response in app.streaming_agent_run_with_events(agent_context):
    print(response)

Event from an unknown agent: AgentSpace_root_agent, event id: 14076651604820872102
Event from an unknown agent: AgentSpace_root_agent, event id: wrWG0v77


{'events': [{'content': {'parts': [{'text': "I'm a large language model, trained by Google.  I'm built using a technique called deep learning, which involves training a neural network on a massive dataset of text and code.  This allows me to understand and generate human-like text, translate languages, and answer your questions in an informative way.\n"}], 'role': 'model'}, 'invocation_id': 'e-7abb5dd0-7935-42cf-ac46-fb55afb96399', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'IBbYjcLm', 'timestamp': 1743476493.185047}]}


In [ ]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"invocation_id":"14076651604820871801","author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for response in app.streaming_agent_run_with_events(agent_context):
    for event in response.get("events", []):
        print(event)

Event from an unknown agent: AgentSpace_root_agent, event id: 14076651604820872102
Event from an unknown agent: AgentSpace_root_agent, event id: kwHUEAI1


{'content': {'parts': [{'text': 'I\'m a large language model, trained by Google.  I don\'t have personal experiences like humans do, so I can\'t describe my "building" in the same way.  My development involved training on a massive dataset of text and code, allowing me to learn patterns and generate human-like text.\n'}], 'role': 'model'}, 'invocation_id': 'e-7653b04d-dd7c-438d-bb2d-f5048c84cee2', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'Yas6wtCN', 'timestamp': 1743476077.147322}


You can navigate to https://pantheon.corp.google.com/traces/explorer;project=reasoning-engine-test-1 to view the traces for it (they might take some time to show up).
* If you don't have the permissions to view them in Cloud Trace, please follow the instructions in https://cloud.google.com/trace/docs/finding-traces#before_you_begin to grant yourself permission to use Trace Explorer with the project.

# Deploy to Vertex AI Agent Engine
To call and manage ADK applications in production, deploy the `app` to Vertex AI Agent Engine.

In [ ]:
remote_app = agent_engines.create(
    app,
    requirements=[
        WHL_FILE,
        "google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226",
    ],
    extra_packages=[
        WHL_FILE,
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.86.0', 'cloudpickle': '3.1.1'}
    google_adk-0.0.2.dev20250326+nightly740999296-py3-none-any.whl
                                ^
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google_adk-0.0.2.dev20250326+nightly740999296-py3-none-any.whl', 'google-cloud-aiplatform[agent_engines] @ git+https://github.com/googleapis/python-aiplatform.git@copybara_738852226', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket reasoning-engine-test-1-bucket
INFO:vertexai.agent_engines:Wrote to gs://reasoning-engine-test-1-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://reasoning-engine-test-1-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://reasoning-eng

**Remark**: You can ignore the following warnings:
* (i) `WARNING:vertexai.agent_engines:Failed to parse constraint: ` and
* (ii) `WARNING:vertexai.agent_engines:The following requirements are missing: {'cloudpickle'}`

We have auto-heal mechanisms to still reflect them in the final list of requirements: `INFO:vertexai.agent_engines:The final list of requirements`

**Remark**: The automatic provisioning of permissions for Vertex AI Sessions have not rolled out yet, so you'll have to follow https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/set-up#service-agent in the meantime to grant "Vertex AI User" (`roles/aiplatform.user`) to the service account in your GCP project

## Let's try it (remotely)

In [ ]:
import pprint

pprint.pprint(remote_app.operation_schemas())

[{'api_mode': '',
  'description': None,
  'name': 'get_session',
  'parameters': {'properties': {'session_id': {'type': 'string'},
                                'user_id': {'type': 'string'}},
                 'required': ['user_id', 'session_id'],
                 'type': 'object'}},
 {'api_mode': '',
  'description': None,
  'name': 'list_sessions',
  'parameters': {'properties': {'user_id': {'type': 'string'}},
                 'required': ['user_id'],
                 'type': 'object'}},
 {'api_mode': '',
  'description': None,
  'name': 'create_session',
  'parameters': {'properties': {'session_id': {'nullable': True,
                                               'type': 'string'},
                                'state': {'nullable': True, 'type': 'object'},
                                'user_id': {'type': 'string'}},
                 'required': ['user_id'],
                 'type': 'object'}},
 {'api_mode': '',
  'description': None,
  'name': 'delete_session',
  'parame

In [ ]:
session1 = remote_app.create_session(user_id="ysian")
session1

{'events': [],
 'last_update_time': 1743476405.532781,
 'state': {},
 'id': '1808138625540423680',
 'app_name': '9206633071833513984',
 'user_id': 'ysian'}

In [ ]:
remote_app.list_sessions(user_id="ysian")

{'session_ids': ['1808138625540423680']}

In [ ]:
session = remote_app.get_session(user_id="ysian", session_id=session1["id"])
session

{'events': [],
 'user_id': 'ysian',
 'state': {},
 'id': '1808138625540423680',
 'app_name': '9206633071833513984',
 'last_update_time': 1743476405.532781}

In [ ]:
for event in remote_app.stream_query(
    user_id="ysian",
    session_id=session1["id"],
    message="roll a 2 sided die",
):
    print(event)

{'parts': [{'function_call': {'id': 'af-3903d72b-6a45-45a5-a626-c733ede546c1', 'args': {'sides': 2}, 'name': 'roll_die'}}], 'role': 'model'}
{'parts': [{'function_response': {'id': 'af-3903d72b-6a45-45a5-a626-c733ede546c1', 'name': 'roll_die', 'response': {'result': 1}}}], 'role': 'user'}
{'parts': [{'text': 'I rolled a 2 sided die and got 1.\n'}], 'role': 'model'}


In [ ]:
for event in remote_app.stream_query(
    user_id="ysian",
    session_id=session1["id"],
    message="What were the previous rolls?",
):
    print(event)

{'parts': [{'text': 'The only roll so far was a 1 on a 2 sided die.\n'}], 'role': 'model'}


In [ ]:
agent_context = '{"message":{"role":"user","parts":[{"text":"How were you built?"}]},"events":[{"content":{"role":"user","parts":[{"text":"how were you built ?"}]},"invocation_id":"1407665160482087180","author":"AgentSpace_root_agent"},{"content":{"role":"model","parts":[{"functionCall":{"name":"agentspaceak","args":{"question":"How were you built?"},"id":"14076651604820872102"}}]},"invocation_id":"14076651604820871801","author":"AgentSpace_root_agent","id":"14076651604820872102"}]}'

for event in remote_app.streaming_agent_run_with_events(request_json=agent_context):
    print(event)

{'events': [{'content': {'parts': [{'text': "I'm a large language model, trained by Google.  I'm built using a massive dataset of text and code, and trained using techniques like deep learning.\n"}], 'role': 'model'}, 'invocation_id': 'e-b664b565-4c40-430b-bac5-2ed963a5a7fb', 'author': 'data_processing_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '7lIOJS7j', 'timestamp': 1743477026.393639}]}


# AgentSpace integration

First, submit request to @krzysiekw with your GCP project number and Agentspace app URL
https://docs.google.com/document/d/1aNIgYM_CphUY4ay8oTn3yxUtRSTcn-TSNYkH2jdlyFs

Next, update config to link Agent to Agentspace:

In [ ]:
%%bash
export PROJECT_ID="ucs-ga-fishfood-1"
export PROJECT_NUMBER="862721868538"
export REASONING_ENGINE="projects/862721868538/locations/us-central1/reasoningEngines/4056344685470285824"
export AGENT_DISPLAY_NAME="Test Agent"
export AGENT_DESCRIPTION="The agent can only answer how it was built."
export AGENT_ID="test_agent"
export AS_APP="itsm-agent-dogfood_1742417292379"

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'

Finally, try the Agentspace UI:

1. Open up the Agentspace app in GCP console
2. From left menu, click `Integration`
3. Open the URL provided
4. From left menu, select the agent deployed

# Clean up

In [ ]:
remote_app.delete(force=True)

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/349765473021/locations/us-central1/operations/6970185842086117376
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/349765473021/locations/us-central1/reasoningEngines/9206633071833513984
